In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList
from qiskit_aer.primitives import Estimator, Sampler

from circuit_knitting.cutting import (
    partition_problem,
    reconstruct_expectation_values,
)

In [2]:
circuit = EfficientSU2(4, entanglement="linear", reps=2).decompose()
circuit.assign_parameters([0.8] * len(circuit.parameters), inplace=True)
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])

In [3]:
partitioned_problem = partition_problem(
    circuit=circuit,
    partition_labels="AABB",
    num_samples=np.inf,
    observables=observables,
)
partitioned_problem._fields

('subcircuits', 'subexperiments', 'weights', 'bases', 'observables')

In [4]:
from circuit_knitting.utils.simulation import ExactSampler
sampler_a = ExactSampler()
sampler_b = ExactSampler()

quasi_dists_a = (
    sampler_a.run(partitioned_problem.subexperiments["A"]).result().quasi_dists
)
quasi_dists_b = (
    sampler_b.run(partitioned_problem.subexperiments["B"]).result().quasi_dists
)

In [5]:
quasi_dists = {"A": quasi_dists_a, "B": quasi_dists_b}
simulated_expvals = reconstruct_expectation_values(
    partitioned_problem,
    quasi_dists,
)
simulated_expvals

[0.1715361328315974,
 0.1815846003933743,
 0.3095869136119664,
 0.4403603630080304,
 0.0817303685507042,
 0.7062381543224684]